- オリジナルインプット
  プレーンテキストで、TITLE: xxx　と REVIEW: yyy　の2つから成る

逆順
- sequential_chain = SimpleSequentialChain(chains=[transform_chain,summary_chain], verbose=True)でtransforn_chainとsummary_chainが順次で呼ばれる
- summary_chain定義
  
  サマリーを作成してというリクエスト
- transform_chain = TransFormChain()
  
  CBはparam transform_cb: Callable[[Dict[str, str]], Dict[str, str]] [Required] (alias 'transform')なのでキーをinput_variablesで定義('text)

- trasformer_fun(inputs:dict) -> dict:定義

  プレーンテキストを受け取るのにdictでキー'text'経由で受け取る。

inputは'text'で、outputは'output'。

  
  

In [1]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
yelp_review = open('../langchain-course-main/June 2024 Update/02-Chains/yelp_review.txt').read()

In [3]:
print(type(yelp_review),"\n")
print(yelp_review)

<class 'str'> 

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, 

In [4]:
def transformer_fun(inputs: dict) -> dict:
    '''
    Notice how this always takes an inputs dictionary.
    Also outputs a dictionary. You can call the output and input keys whatever you want, 
    just make sure to reference it correct in the chain call.
    '''
    # GRAB INCOMING CHAIN TEXT
    text = inputs['text']
    print("---------------------------------")
    print(inputs.keys())
    print("---------------------------------")
    print(type(text))
    print("---------------------------------")
    print(text)
    print("---------------------------------")
    only_review_text = text.split('REVIEW:')[-1]
    lower_case_text = only_review_text.lower()
    return {'output':lower_case_text}

In [5]:
transform_chain = TransformChain(input_variables=['text'],
                                 output_variables=['output'],
                                 transform=transformer_fun)

In [6]:
template = "Create a one sentence summary of this review:\n{review_text}"

In [7]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
# llm = ChatOpenAI()
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # type: ignore
prompt = ChatPromptTemplate.from_template(template)
summary_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="review_summary")

In [9]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain,summary_chain],
                                        verbose=True)




In [ ]:
result = sequential_chain(yelp_review)

In [36]:
result['output']

'This restaurant offers an unforgettable dining experience with stunning ambiance, exceptional food, and impeccable service, making it a culinary haven for food lovers. \n'

In [37]:
result['input']

"TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!\n\nREVIEW:\nOH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!\n\nFIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.\n\nNOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!\n\nTHEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!\n\nTHE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND 

In [38]:
print(type(result))
print(result.keys())

<class 'dict'>
dict_keys(['input', 'output'])
